In [79]:
import jinja2
import json
env = jinja2.Environment('{%', '%}', '{', '}')

class kl_object(dict):
    def __init__(self, data):
        dict.__init__(self)
        if isinstance(data, str):
            data = json.loads(data.strip())
        for k,v in data.items():
            if isinstance(v, dict):
                self[k] = kl_object(v)
            elif isinstance(v, list) and len(v) == 4:
                self[k] = kl_object({ "x":v[0], "y":v[1], "z":v[2] })
            else:
                self[k] = v
    def __getattr__(self, key):
        return self[key]

printer = kl_object({ # add items you need, see https://www.klipper3d.org/Status_Reference.html 
    "angle" : {},
    "bed_mesh" : {},
    "bed_screws" : {},
    "configfile" : {
        "config" : {
            "stepper_y"  : {"enable_pin":"!PF14","step_pin":"PF13","dir_pin":"PF12","rotation_distance":"40","microsteps":"32","full_steps_per_rotation":"200","endstop_pin":"PG13","position_min":"0","position_endstop":"300","position_max":"300","homing_speed":"25","homing_retract_dist":"5","homing_positive_dir":"true"},
            "stepper_x"  : {"enable_pin":"!PF15","step_pin":"PG0","dir_pin":"PG1","rotation_distance":"40","microsteps":"32","full_steps_per_rotation":"200","endstop_pin":"PG12","position_min":"0","position_endstop":"300","position_max":"300","homing_speed":"25","homing_retract_dist":"5","homing_positive_dir":"true"},
            "stepper_z"  : {"enable_pin":"!PF1","step_pin":"PC13","dir_pin":"PF0","endstop_pin":"PG15","rotation_distance":"4","microsteps":"32","position_max":"250","position_min":"-2.5","homing_speed":"8.0","second_homing_speed":"3","homing_retract_dist":"3","position_endstop":"0.635"},
            "stepper_z1" : {"enable_pin":"!PG2","step_pin":"PF9","dir_pin":"PF10","rotation_distance":"4","microsteps":"32"},"stepper_z2":{"enable_pin":"!PA0","step_pin":"PG4","dir_pin":"PC1","rotation_distance":"4","microsteps":"32"},
        },
        "settings" : {},
        "warnings" : {},
        "save_config_pending" : False,
        "save_config_pending_items" : {},
    },
    "display_status" : {},
    "endstop_phase" : {},
    "exclude_object" : {},
    "extruder_stepper" : {},
    "fan" : {},
    "filament_switch_sensor" : {},
    "filament_motion_sensor" : {},
    "firmware_retraction" : {},
    "gcode" : {},
    "gcode_button" : {},
    "gcode_macro" : {},
    "gcode_move" : kl_object('{"speed_factor":1.0,"speed":1500.0,"extrude_factor":1.0,"absolute_coordinates":true,"absolute_extrude":true,"homing_origin":[0.0,0.0,0.0,0.0],"position":[0.0,0.0,0.0,0.0],"gcode_position":[0.0,0.0,0.0,0.0]}'),
    "hall_filament_width_sensor" : {},
    "heater" : {},
    "heaters" : {},
    "idle_timeout" : {},
    "led" : {},
    "manual_probe" : {},
    "mcu" : {},
    "motion_report" : {},
    "output_pin" : {},
    "palette2" : {},
    "pause_resume" : {},
    "print_stats" : {},
    "probe" : kl_object({
        "name"  : "",
        "last_query" : True,
        "last_z_result" : -0.1234,
    }),
    "pwm_cycle_time" : {},
    "quad_gantry_level" : {},
    "query_endstops" : {},
    "screws_tilt_adjust" : {},
    "servo" : {},
    "stepper_enable" : {},
    "system_stats" : {},
    "temperature sensors" : {},
    "temperature_fan" : {},
    "temperature_sensor" : {},
    "tmc drivers" : {},
    "toolhead" : {},
    "dual_carriage" : {},
    "virtual_sdcard" : {},
    "webhooks" : {},
    "z_thermal_adjust" : {},
    "z_tilt" : {},
})      

In [80]:
print(printer["gcode_move"].homing_origin)
print(printer.gcode_move.homing_origin.__dict__)
print(printer.gcode_move.homing_origin.x)
print(printer.configfile.config["stepper_x"])
print("position_min" in printer.configfile.config["stepper_x"])

{'x': 0.0, 'y': 0.0, 'z': 0.0}
{}
0.0
{'enable_pin': '!PF15', 'step_pin': 'PG0', 'dir_pin': 'PG1', 'rotation_distance': '40', 'microsteps': '32', 'full_steps_per_rotation': '200', 'endstop_pin': 'PG12', 'position_min': '0', 'position_endstop': '300', 'position_max': '300', 'homing_speed': '25', 'homing_retract_dist': '5', 'homing_positive_dir': 'true'}
True


In [81]:
args = {
    "params" : {
        "X" : "0,100",
        "Y" : "0,200",
        "Z" : "0,10",
    },
    'printer' : printer,
}
macro = """
  # Transform to G53 coordinates
  {% set x_min = params.X.split(",")[0]|float - printer.gcode_move.homing_origin.x|float %}
  {% set y_min = params.Y.split(",")[0]|float - printer.gcode_move.homing_origin.y|float %}
  {% set z_min = params.Z.split(",")[0]|float - printer.gcode_move.homing_origin.z|float %}
  {% set x_max = params.X.split(",")[1]|float - printer.gcode_move.homing_origin.x|float %}
  {% set y_max = params.Y.split(",")[1]|float - printer.gcode_move.homing_origin.y|float %}
  {% set z_max = params.Z.split(",")[1]|float - printer.gcode_move.homing_origin.z|float %}

  # Get machine limits from config
  {% if "position_min" in printer.configfile.config["stepper_x"] %}
    {% set x_printable_min = printer.configfile.config["stepper_x"].position_min|float %}
  {% else %}
    {% set x_printable_min = 0|float %}
  {% endif %}
  {% if "position_min" in printer.configfile.config["stepper_y"] %}
    {% set y_printable_min = printer.configfile.config["stepper_y"].position_min|float %}
  {% else %}
    {% set y_printable_min = 0|float %}
  {% endif %}
  {% if "position_min" in printer.configfile.config["stepper_z"] %}
    {% set z_printable_min = printer.configfile.config["stepper_z"].position_min|float %}
  {% else %}
    {% set z_printable_min = 0|float %}
  {% endif %}  
  {% set x_printable_max = printer.configfile.config["stepper_x"].position_max|float %}
  {% set y_printable_max = printer.configfile.config["stepper_y"].position_max|float %}
  {% set z_printable_max = printer.configfile.config["stepper_z"].position_max|float %}

  # TODO: compare both values. 
  #       If there is a point outside the printable area stop NOW instead of later!
  RESPOND MSG="TODO: check if x,y,z are inside the machineable area"
  RESPOND MSG=" "
  RESPOND MSG="Parameter:"
  RESPOND MSG="x = {params.X}"
  RESPOND MSG="y = {params.Y}"
  RESPOND MSG="z = {params.Z}"
  RESPOND MSG=" "

  RESPOND MSG="Absolute coordinates:"
  RESPOND MSG="x = {x_min}, {x_max}"
  RESPOND MSG="y = {y_min}, {y_max}"
  RESPOND MSG="z = {z_min}, {z_max}"
  RESPOND MSG=" "

  RESPOND MSG="Printable area:"
  RESPOND MSG="x = {x_printable_min}, {x_printable_max}"
  RESPOND MSG="y = {y_printable_min}, {y_printable_max}"
  RESPOND MSG="z = {z_printable_min}, {z_printable_max}"
  RESPOND MSG=" "

  RESPOND MSG="Printable?"
  RESPOND MSG="x = {x_printable_min <= x_min and x_max <=  x_printable_max}"
  RESPOND MSG="y = {y_printable_min <= y_min and y_max <=  y_printable_max}"
  RESPOND MSG="z = {z_printable_min <= z_min and z_max <=  z_printable_max}"
  RESPOND MSG=" "
"""

In [82]:
gcode = env.from_string(macro).render(args)
gcode = "\n".join([i.strip() for i in gcode.strip().splitlines()])

lines = gcode.splitlines()
for i in range(len(lines)-2, 0, -1):
    if lines[i+1] == "" and lines[i] == "":
        del lines[i+1]
gcode = "\n".join(lines)

# copy to clipboard
import pyperclip
pyperclip.copy(gcode)

# Gcode is pasted into your clipboard! 
<iframe src='https://nraynaud.github.io/webgcode/' width="100%" height="800"></iframe>